In [1]:
# coding=utf-8
from datetime import datetime, timedelta
from datetime import time
import pandas as pd
import pandahouse

In [2]:
connection_1 = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : 'dpo_python_2020', 
    'user' : 'student', 
    'database' : 'simulator'
}

In [3]:
query = """SELECT  toDate(time) as event_date,
                   countIf(action='like') AS likes,
                   countIf(action='view') AS views,
                   user_id, 
                   gender,
                   os, 
                   age
            FROM simulator_20240620.feed_actions
            WHERE toDate(time) = today() - 1
            GROUP BY user_id, gender, os, age, event_date
            """

df_feed_actions = pandahouse.read_clickhouse(connection=connection_1, query=query)

In [4]:
query = """SELECT
                toDate(time) as event_date,
                received.messages_received,
                COUNT(*) AS messages_sent,
                COUNT(DISTINCT receiver_id) AS users_received,
                received.users_sent,
                user_id,
                gender,
                os,
                age
            FROM simulator_20240620.message_actions 
                LEFT JOIN
                (
                    SELECT
                        receiver_id,
                        toDate(time) as event_date,
                        COUNT(*) AS messages_received,
                        COUNT(DISTINCT user_id) AS users_sent
                    FROM simulator_20240620.message_actions
                    WHERE toDate(time) = today() - 1
                    GROUP BY receiver_id, event_date
                ) AS received
                ON
                    message_actions.user_id = received.receiver_id
                WHERE toDate(time) = today() - 1
                GROUP BY user_id, received.messages_received, received.users_sent, event_date, gender, os, age
                ORDER BY user_id"""
df_message_actions = pandahouse.read_clickhouse(connection=connection_1, query=query)

In [5]:
df_message_actions

,event_date,messages_received,messages_sent,users_received,users_sent,user_id,gender,os,age
0,2024-07-25,5,2,2,4,214,0,Android,21
1,2024-07-25,10,10,10,10,231,1,Android,26
2,2024-07-25,2,8,2,2,266,0,Android,34
3,2024-07-25,1,1,1,1,377,1,Android,39
4,2024-07-25,0,6,6,0,577,0,Android,21
...,...,...,...,...,...,...,...,...,...
2133,2024-07-25,0,6,6,0,131502,1,Android,20
2134,2024-07-25,0,4,2,0,131503,1,Android,38
2135,2024-07-25,0,15,14,0,131511,1,Android,26
2136,2024-07-25,0,9,2,0,131521,1,Android,59


In [6]:
merged_df = pd.merge(df_feed_actions, df_message_actions, on=['user_id', 'event_date', 'gender', 'os', 'age'], how='inner')

In [7]:
df_gender = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
df_gender['dimension'] = 'gender'
df_gender['dimension_value'] = merged_df['gender']

df_age = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
df_age['dimension'] = 'age'
df_age['dimension_value'] = merged_df['age'].astype(str)

df_os = merged_df[['event_date', 'likes', 'views', 'messages_sent', 'messages_received', 'users_sent', 'users_received']].copy()
df_os['dimension'] = 'os'
df_os['dimension_value'] = merged_df['os']

# Объединяем все DataFrame с помощью pd.concat
df_expanded = pd.concat([df_gender, df_age, df_os], ignore_index=True)

In [8]:
df_expanded_os = df_expanded.loc[df_expanded.dimension == 'os'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .sum()

In [164]:
df_expanded_os

,event_date,dimension,dimension_value,likes,views,messages_sent,messages_received,users_sent,users_received
0,2024-07-24,os,Android,1834,8202,1142,1122,911,881
1,2024-07-24,os,iOS,895,4212,609,828,533,442


In [9]:
df_expanded_gender = df_expanded.loc[df_expanded.dimension == 'gender'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .sum()

df_gender.loc[df_gender['dimension_value'] == 0, 'dimension_value'] = 'female'
df_gender.loc[df_gender['dimension_value'] == 1, 'dimension_value'] = 'male'

In [10]:
df_expanded_age = df_expanded.loc[df_expanded.dimension == 'age'].groupby(['event_date', 'dimension', 'dimension_value'], as_index=False) \
                                              .sum()

In [39]:
df_final = pd.concat([df_expanded_age, df_expanded_gender, df_expanded_os], ignore_index=True)

In [40]:
df_final

,event_date,dimension,dimension_value,likes,views,messages_sent,messages_received,users_sent,users_received
0,2024-07-25,age,14,5,24,2,1,1,1
1,2024-07-25,age,15,55,266,28,8,7,18
2,2024-07-25,age,16,72,300,27,24,23,24
3,2024-07-25,age,17,123,527,100,39,38,89
4,2024-07-25,age,18,143,648,73,57,57,73
5,2024-07-25,age,19,104,434,74,117,112,70
6,2024-07-25,age,20,192,1002,129,248,199,125
7,2024-07-25,age,21,160,756,130,147,141,128
8,2024-07-25,age,22,157,674,110,163,159,109
9,2024-07-25,age,23,66,325,61,73,70,60


In [169]:
pd.to_datetime(df_final['event_date'], unit='ns')

0    2024-07-24
1    2024-07-24
2    2024-07-24
3    2024-07-24
4    2024-07-24
5    2024-07-24
6    2024-07-24
7    2024-07-24
8    2024-07-24
9    2024-07-24
10   2024-07-24
11   2024-07-24
12   2024-07-24
13   2024-07-24
14   2024-07-24
15   2024-07-24
16   2024-07-24
17   2024-07-24
18   2024-07-24
19   2024-07-24
20   2024-07-24
21   2024-07-24
22   2024-07-24
23   2024-07-24
24   2024-07-24
25   2024-07-24
26   2024-07-24
27   2024-07-24
28   2024-07-24
29   2024-07-24
30   2024-07-24
31   2024-07-24
32   2024-07-24
33   2024-07-24
34   2024-07-24
35   2024-07-24
36   2024-07-24
37   2024-07-24
38   2024-07-24
39   2024-07-24
40   2024-07-24
41   2024-07-24
42   2024-07-24
43   2024-07-24
44   2024-07-24
45   2024-07-24
46   2024-07-24
47   2024-07-24
Name: event_date, dtype: datetime64[ns]

In [34]:
df_final['event_date'] = df_final['event_date'] + pd.Timedelta(hours=5, minutes=10, seconds=3)

In [18]:
df_final.dtypes

event_date           datetime64[ns]
dimension                    object
dimension_value              object
likes                        uint64
views                        uint64
messages_sent                uint64
messages_received            uint64
users_sent                   uint64
users_received               uint64
dtype: object

In [19]:
connection_2 = { 
    'host' : 'https://clickhouse.lab.karpov.courses', 
    'password' : '656e2b0c9c', 
    'user' : 'student-rw', 
    'database' : 'test'
}

In [35]:
        #запрос создания таблички имеет примерно следующую структуру
        query_test = '''CREATE TABLE IF NOT EXISTS test.shaimuirat_task_7
                        (event_date DateTime64,
                        dimension String,
                        dimension_value String,
                        views UInt64,
                        likes UInt64,
                        messages_received UInt64,
                        messages_sent UInt64,
                        users_received UInt64,
                        users_sent UInt64
                        )
                        ENGINE = MergeTree()
                        ORDER BY event_date
                '''

In [36]:
pandahouse.execute(query_test, connection=connection_2)

b''

In [41]:
pandahouse.to_clickhouse(df=df_final, table="shaimuirat_task_7", index=False, connection=connection_2)

ClickhouseException: b'Code: 27. DB::ParsingException: Cannot parse input: expected \'"\' before: \'14",5,24,2,1,1,1\\r\\n"2024-07-25","age","15",55,266,28,8,7,18\\r\\n"2024-07-25","age","16",72,300,27,24,23,24\\r\\n"2024-07-25","age","17",123,527,100,39,38,89\\r\\n"2024-07-2\': \nRow 1:\nColumn 0,   name: event_date,        type: DateTime64(3), parsed text: "<DOUBLE QUOTE>2024-07-25<DOUBLE QUOTE>,<DOUBLE QUOTE>age<DOUBLE QUOTE>,<DOUBLE QUOTE>"ERROR\nCode: 27. DB::ParsingException: Cannot parse input: expected \'"\' before: \'14",5,24,2,1,1,1\\r\\n"2024-07-25","age","15",55,266,28,8,7,18\\r\\n"2024-07-25","age","16",72,300,27,24,23,24\\r\\n"2024-07-25","age","17",123,527,100,39,38,89\\r\\n"2024-07-2\'. (CANNOT_PARSE_INPUT_ASSERTION_FAILED) (version 21.12.2.17 (official build))\n\n: While executing ParallelParsingBlockInputFormat: (at row 1)\n. (CANNOT_PARSE_INPUT_ASSERTION_FAILED) (version 21.12.2.17 (official build))\n'

In [92]:
df_final = df_final.convert_dtypes()

In [ ]:
['dimension_value']